In [20]:
from psycopg2 import connect
import configparser
import pandas as pd

# Known issues
* if train doesn't stay at the station for enough time to get a request where it's `AtStation` we miss that data
* station to station metrics should only look at consecutive stations

# Create connection and view sample data

In [14]:
CONFIG = configparser.ConfigParser(interpolation=None)
CONFIG.read('./ttc_db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [16]:
sql = '''
SELECT *
FROM ntas_data
LIMIT 1
'''

with con.cursor() as cur:
    cur.execute(sql)
    colnames = (desc[0] for desc in cur.description)
    values = cur.fetchone()
    print(list(zip(colnames, values)))

[('requestid', 1), ('id', 32909720863), ('station_char', 'SHW1'), ('subwayline', 'YUS'), ('system_message_type', 'Normal'), ('timint', Decimal('0.0')), ('traindirection', 'North'), ('trainid', 132), ('train_message', 'AtStation'), ('train_dest', 'Southbound<br/> To Union')]


# Create materialized views

### `arrival_time`: filters only trains that have just arrived at a station
Columns: `subwayline`,`station_char`,`trainid`,`arrive_time`

In [17]:
sql = '''
DROP MATERIALIZED VIEW IF EXISTS arrival_times CASCADE; 
CREATE MATERIALIZED VIEW arrival_times AS 
WITH before_after AS (
SELECT subwayline, station_char, trainid,
train_message, create_date,
LAG(train_message) OVER (PARTITION BY subwayline, station_char, trainid ORDER BY create_date) as previous_message,
FROM ntas_data
JOIN requests USING (requestid)
)
SELECT subwayline, station_char, trainid, create_date as arrive_time
FROM before_after
WHERE train_message = 'AtStation'
AND previous_message = 'Arriving'
''' 
with con:
    with con.cursor() as cur:
        cur.execute(sql)

### `trip_metrics`: station to station trip metrics
Columns: `subwayline`,`station_char`,`next_station`,`trips`,`fastest_trip`,`avg_duration`,`median_duration`

Additional cleaning:
* next station can't be the same as current one
* duration between 10 seconds and one hour
* combination of stations needs to show up at least 10 times

In [19]:
sql = '''
DROP MATERIALIZED VIEW IF EXISTS trip_metrics CASCADE; 
CREATE MATERIALIZED VIEW trip_metrics AS 
  WITH next_stations AS (
    SELECT subwayline, station_char, trainid, arrive_time,
    LEAD(station_char) OVER (PARTITION BY subwayline, trainid ORDER BY arrive_time) as next_station,
    LEAD(arrive_time) OVER (PARTITION BY subwayline, trainid ORDER BY arrive_time) as next_arrival
    FROM arrival_times
  ), durations AS (
    SELECT subwayline, station_char, next_station, trainid, arrive_time, EXTRACT(EPOCH FROM (next_arrival::timestamp - arrive_time::timestamp)) as duration
    FROM next_stations
    WHERE next_arrival IS NOT NULL
    AND station_char <> next_station
    AND EXTRACT(EPOCH FROM (next_arrival::timestamp - arrive_time::timestamp)) BETWEEN 10 AND 3600
)
  SELECT DISTINCT subwayline, station_char, next_station,
  COUNT(1) as trips,
  MIN(duration) as fastest_trip,
  ROUND(AVG(duration)) as average,
  PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY duration) as median_duration
  FROM durations
  GROUP BY 1, 2, 3
  HAVING COUNT(1) > 10
''' 

with con:
    with con.cursor() as cur:
        cur.execute(sql)

# Data exploration

In [24]:
# Shortest trips

sql = '''
SELECT *
FROM trip_metrics
ORDER BY median_duration ASC
LIMIT 5
'''
data = pd.read_sql(sql, con)
data

,subwayline,station_char,next_station,trips,fastest_trip,average,median_duration
0,BD,BAT1,BSP1,3165,11.0,67.0,59.0
1,YUS,STP1,OSG1,646,22.0,63.0,60.0
2,BD,SGL2,BSP2,4965,10.0,70.0,61.0
3,BD,LAN1,DUF1,2687,13.0,64.0,61.0
4,BD,PAP1,DON1,2907,21.0,70.0,61.0


In [27]:
# Longest trips

sql = '''
SELECT *
FROM trip_metrics
WHERE trips > 1000
ORDER BY median_duration DESC
LIMIT 5
'''
data = pd.read_sql(sql, con)
data

,subwayline,station_char,next_station,trips,fastest_trip,average,median_duration
0,YUS,FIN1,SHP2,1075,227.0,469.0,434.0
1,YUS,FIN2,SHP2,1168,177.0,461.0,422.0
2,BD,KEN1,WAR2,2019,175.0,448.0,420.0
3,YUS,CVL1,DUP1,1062,297.0,402.0,413.0
4,BD,KEN2,WAR2,2187,174.0,425.0,411.0
